<a href="https://colab.research.google.com/github/Dario-Zela/Neural-Networks-Test/blob/main/Self_made_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
class Value:

  def __init__(self, value, children = (), label = ""):
    self.data = np.atleast_2d(np.array(value, 'float64'))
    self.grad = np.zeros_like(self.data)
    self._backward = lambda : None
    self._prev = set(children)
    self.label = label

  def __repr__(self):
    s = f"Value {self.label}: Stores {self.data} | Gradients {self.grad}"
    if len(self._prev) != 0:
      s += "\nContains : {"

    for child in self._prev:
      s += f"\n\t{child}"

    if len(self._prev) != 0:
      s += "\n\t}"

    return s

  def st(self):
    return f"Value {self.label}: Stores {self.data} | Gradients {self.grad}"


  def __add__(self, other):
    if not isinstance(other, Value):
      other = Value(other)
      
    out = Value(np.add(self.data, other.data), (self, other), label= f"{self.label}+{other.label}")

    def _backward():
      self.grad += out.grad
      other.grad += out.grad
      #print("add")
      #print(self.grad)

    out._backward = _backward
    return out

  def __mul__(self, other):
    if not isinstance(other, Value):
      other = Value(other)
    out = Value(np.matmul(other.data, self.data.transpose()), (self, other), label= f"{self.label}*{other.label}")

    def _backward():
      self.grad += np.matmul(out.grad.transpose(), other.data)
      other.grad += np.matmul(out.grad, self.data)
      #print("mul")
      #print(self.grad)
    
    out._backward = _backward
    return out

  def tanh(self):
    x = self.data
    t = np.tanh(self.data)

    out = Value(t, (self, ), label= f"tanh({self.label})")

    def _backward():
      self.grad +=  (1 - t ** 2) * out.grad
      #print("tanh")
      #print(x)
      #print(self.grad) 
    
    out._backward = _backward
    return out

  def __sub__(self, other):
    if not isinstance(other, Value):
      other = Value(other)
      
    out = Value(np.subtract(self.data, other.data), (self, other), label= f"{self.label}-{other.label}")

    def _backward():
      #print("sub")
      #print(self.grad)
      self.grad += out.grad
      other.grad -= out.grad
      #print(out.grad)

    out._backward = _backward
    return out

  def backward(self):
    topo = []
    visited = set()

    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)

    build_topo(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

  def square(self):
    out = Value(self.data ** 2, (self, ), label= f"({self.label})**2")
    def _backward():
      #print("square")
      self.grad += 2 * out.grad * self.data
      #print(self.grad)

    out._backward = _backward

    return out

  def zero_grad(self):
    self.grad = 0.0
    for child in self._prev:
      child.zero_grad()

In [ ]:
class Layer:
  def __init__(self, nin, nout):
    self.w = Value(np.random.rand(nout, nin), label = f"w{nin}{nout}")
    self.b = Value(np.random.rand(nout), label = f"b{nin}{nout}")

  def __call__(self, x):
    a = (self.w * x + self.b).tanh()
    return a
    

  def params(self):
    return [self.w, self.b]

class MLP:
  def __init__(self, nin, nouts):
    sz = [nin] + nouts
    self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]

  def __call__(self, x):
    x = np.atleast_2d(np.array(x))
    for layer in self.layers:
      x = layer(x)
    return x
  
  def params(self):
    return [p for layer in self.layers for p in layer.params()]
  
  def zero_grad(self):
    for p in self.params():
      p.zero_grad()
    

In [ ]:
ys = [1,-1,-1,1]
xs = [
    [2, 3, -1],
    [3, -1, 0.5],
    [0.5,1,1],
    [1,-1,-1]
    ]

In [ ]:
n = MLP(3, [4,3,1])

In [ ]:
for i in range(200):

  ypred = [n(x) for x in xs]

  loss = Value([0])
  for j in ((yout - ygt).square() for ygt, yout in zip(ys, ypred)):
    loss += j

  n.zero_grad()
  loss.backward()

  for p in n.params():
    p.data += -0.05 * p.grad

  print(i, loss.data)


0 [[0.01922368]]
1 [[0.01896089]]
2 [[0.01870502]]
3 [[0.0184558]]
4 [[0.01821297]]
5 [[0.0179763]]
6 [[0.01774556]]
7 [[0.01752053]]
8 [[0.017301]]
9 [[0.01708678]]
10 [[0.01687768]]
11 [[0.01667351]]
12 [[0.01647412]]
13 [[0.01627933]]
14 [[0.01608899]]
15 [[0.01590295]]
16 [[0.01572107]]
17 [[0.0155432]]
18 [[0.01536923]]
19 [[0.01519903]]
20 [[0.01503247]]
21 [[0.01486944]]
22 [[0.01470983]]
23 [[0.01455354]]
24 [[0.01440047]]
25 [[0.01425051]]
26 [[0.01410357]]
27 [[0.01395957]]
28 [[0.01381842]]
29 [[0.01368004]]
30 [[0.01354434]]
31 [[0.01341125]]
32 [[0.0132807]]
33 [[0.01315261]]
34 [[0.01302691]]
35 [[0.01290355]]
36 [[0.01278245]]
37 [[0.01266356]]
38 [[0.01254682]]
39 [[0.01243216]]
40 [[0.01231954]]
41 [[0.0122089]]
42 [[0.01210019]]
43 [[0.01199336]]
44 [[0.01188836]]
45 [[0.01178515]]
46 [[0.01168368]]
47 [[0.01158391]]
48 [[0.01148579]]
49 [[0.01138929]]
50 [[0.01129437]]
51 [[0.01120098]]
52 [[0.0111091]]
53 [[0.01101868]]
54 [[0.01092969]]
55 [[0.0108421]]
56 [[0.0107

In [ ]:
n([1,-1,-1]).st()

'Value tanh(w31*tanh(w43*tanh(w34*+b34)+b43)+b31): Stores [[0.96973147]] | Gradients [[0.]]'